<a href="https://colab.research.google.com/github/Likelipop/Introduction-to-ML/blob/main/%5BNMMH%20HK2%2024-25%5D_22110039_Tr%E1%BA%A7n%20Ti%E1%BA%BFn%20%C4%90%E1%BA%A1t.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>